In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def stemAndRemovePuncAndStopWords(data):
    
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    sno = nltk.stem.SnowballStemmer('english')
    
    for tweet in data:
        for word in tokenizer.tokenize(tweet):
            sno.stem(word)
    
    
    return [" ".join([sno.stem(word) for word in tokenizer.tokenize(tweet)]for tweet in data)] 

#if word not in nltk.corpus.stopwords.words('english')

In [3]:
#Read in the data to a pandas DataFrame
train = pd.read_csv("Tweets/tweet-even-train.csv", encoding="utf-16")
test = pd.read_csv("Tweets/tweet-even-test.csv", encoding="utf-16")

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [10]:
def buildFeatureSet(data):
    vectorizer = CountVectorizer(
        lowercase=True,
        stop_words=None,
        max_df=25,
        min_df=1,
        max_features=None,
        binary=True,
        ngram_range=(2,2)
    )

    features = vectorizer.fit_transform(data['text'].values.astype('U')).toarray()
    feature_names = vectorizer.get_feature_names()
    print("Vocab size: {}".format(len(feature_names)))

    return list(zip(
        [  # zip a list of vocab dicts with their genre
            {
                word: bool(features[row][idx])  # A word and its presence in this song
                for idx, word in enumerate(feature_names)  # Go through all the words in the vocab
            } for row in range(len(features))  # Each dict has |vocab| entries: true or false if the song has that word
        ],data['id']
    ))

In [11]:
train = buildFeatureSet(train)
test = buildFeatureSet(test)

Vocab size: 52334


Vocab size: 15332


In [12]:
classifier = nltk.NaiveBayesClassifier.train(train)

classifier.show_most_informative_features()
nltk.classify.accuracy(classifier, test)

Most Informative Features


               check out = True           Not Tr : Trump  =     16.6 : 1.0
                 see you = True            Trump : Not Tr =     16.1 : 1.0
               are going = True            Trump : Not Tr =     15.4 : 1.0
            meeting with = True            Trump : Not Tr =     14.1 : 1.0
            this morning = True            Trump : Not Tr =     14.1 : 1.0
               you agree = True           Not Tr : Trump  =     13.9 : 1.0
               watch the = True           Not Tr : Trump  =     13.2 : 1.0
               great job = True            Trump : Not Tr =     12.8 : 1.0
                it https = True           Not Tr : Trump  =     12.5 : 1.0
                hope you = True           Not Tr : Trump  =     11.9 : 1.0


0.8902243589743589